In [1]:
##update the metabolite ids and the reaction ids to contain only _c and not _c0
import cobra
from cobra import Metabolite, Reaction, Gene

/usr/local/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#load the most recent version of the model
pst = cobra.io.read_sbml_model("../results/reconstructions/v4_with_all_annotations.xml")

In [3]:
#remove all exterior reactions, aka the exchange and supp biomass equations
#without removal, metabolites from these will interfere with code later, due to updating the metabolite list, errors include "metabolite already exists" but does not pass it
pst.reactions.PAO1_Biomass.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v13.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v13ub.remove_from_model(remove_orphans = True)
pst.reactions.PA_Biomass_v4.remove_from_model(remove_orphans = True)

rxns_to_remove = []
for reaction in pst.reactions:
    if "LPAREN" in reaction.id:
        rxns_to_remove.append(reaction)
    if "PJT" in reaction.id:
        rxns_to_remove.append(reaction)

# remove the flagged reactions. Removing them inside the for loop above can lead to some reactions being skipped due to the list being modified
# during the execution of the for loop.
for rxn in rxns_to_remove:
    rxn.remove_from_model(remove_orphans = True)

In [4]:
#manual check for the reactions and metabolites that may still be present
#if the previous chunk is exicuted multiple times, to where only 1 "yes" appears twice in a row, both of these should print nothing
for reaction in pst.reactions:
    if "LPAREN" in reaction.id:
        print (reaction)
for reaction in pst.reactions:
    if "PJT" in reaction.id:
        #print(reaction)
        print (reaction.id)

In [5]:
#call all of the metabolite ids and reduce the names to only _c or _e
#this is done to standardize the metabolite names, as previously there was a mix of _c and _c0
#the repeated_ids list should render blank
repeated_ids = []

for met in pst.metabolites:
    metid= met.id
    if '_c0' in metid:
        metid = metid[:-1]
        if metid not in pst.metabolites:
            met.id = metid
        else:
            repeated_ids.append(metid)
    if '_e0' in metid:
        metid = metid[:-1]
        if metid not in pst.metabolites:
            met.id = metid
        else:
            repeated_ids.append(metid)
#repeated ids are cooming from interferences with the biomass functions
print (repeated_ids)
    

[]


In [6]:
#add back the necessary exchange reactions that were previously deleted
#each exchange reaction is based on whetehr a _e is in the metaboltie descriptor
for met in pst.metabolites:
    if "_e" in met.id:
        reaction = Reaction('EX_' + met.id)
        reaction.name = 'Exchange' + met.id[:-3]
        reaction.subsystem = 'Exchange'
        reaction.lower_bound = 0.
        reaction.upper_bound = 1000.
        reaction.add_metabolites({met:-1})
        pst.add_reactions([reaction])
    else:
        pass

In [7]:
#add back the appropriate biomass function with its metabolites, this will be biomass v4, eventually
#I will hold off on adding this for now due to the presence of manually added metabolites that do not have the proper seed id
cobra.io.write_sbml_model(pst, "../results/suffix_fixed_no_biomass.xml")